In [38]:
import os
import json
import boto3

import pandas as pd
import plotly.express as px

import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

ACCESS_KEY = os.getenv("ID_KEY_ACCESS")
ACCESS_KEY_SECRETE = os.getenv("KEY_SECRET")
BUCKET_NAME = os.getenv("BUCKETNAME")

Python-dotenv could not parse statement starting at line 5


In [39]:
df_1 = pd.read_json("./hotel_1.json")
df_2 = pd.read_json("./hotel_2.json")
df_3 = pd.read_json("./temps.json")

In [40]:
# merge des 3 dataframe
def merge_outer(data_1:pd.DataFrame, data_2:pd.DataFrame, on:str) -> pd.DataFrame:
    df = pd.merge(data_1, data_2, how='outer', on=on)
    return df

df = merge_outer(df_1, df_2, "hotel")
df = merge_outer(df, df_3, "id")

In [41]:
# drop des hotel ou il manque la note
df.dropna()

,id,ville,hotel,url,note,description,latitutde,longitude,temperature,temps,lat,long
0,1,Le Mont-Saint-Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0",Vous pouvez bénéficier d'une réduction Genius ...,48.614700,-1.509617,"[12.34, 12.43, 11.24, 11.02, 9.62]","[légère pluie, légère pluie, couvert, légère p...",48.635523,-1.510257
1,1,Le Mont-Saint-Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",Installé dans des espaces verts à seulement 2 ...,48.614247,-1.510545,"[12.34, 12.43, 11.24, 11.02, 9.62]","[légère pluie, légère pluie, couvert, légère p...",48.635523,-1.510257
2,1,Le Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,0",Vous pouvez bénéficier d'une réduction Genius ...,48.635085,-1.510540,"[12.34, 12.43, 11.24, 11.02, 9.62]","[légère pluie, légère pluie, couvert, légère p...",48.635523,-1.510257
3,1,Le Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",L'hôtel De La Digue est un établissement tradi...,48.616882,-1.510918,"[12.34, 12.43, 11.24, 11.02, 9.62]","[légère pluie, légère pluie, couvert, légère p...",48.635523,-1.510257
4,1,Le Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",Vous pouvez bénéficier d'une réduction Genius ...,48.635349,-1.510379,"[12.34, 12.43, 11.24, 11.02, 9.62]","[légère pluie, légère pluie, couvert, légère p...",48.635523,-1.510257
...,...,...,...,...,...,...,...,...,...,...,...,...
870,35,"1er arr., Lyon",Hamac Suites - Le Plâtre - Place des Terreaux,https://www.booking.com/hotel/fr/hamac-suites-...,"8,7","Doté d'une connexion Wi-Fi gratuite, le Hamac ...",45.766106,4.833282,"[12.04, 12.71, 12.54, 10.15, 11.62]","[partiellement nuageux, couvert, légère pluie,...",46.159113,-1.152043
871,35,"7e arr., Lyon",B&B HOTEL LYON Centre Perrache Berthelot,https://www.booking.com/hotel/fr/b-amp-b-lyon-...,"7,8","Ouvert en décembre 2017, le B&B HOTEL LYON Cen...",45.747809,4.835540,"[12.04, 12.71, 12.54, 10.15, 11.62]","[partiellement nuageux, couvert, légère pluie,...",46.159113,-1.152043
872,35,"2e arr., Lyon",Hôtel Charlemagne,https://www.booking.com/hotel/fr/hotelbestwest...,"7,8",Vous pouvez bénéficier d'une réduction Genius ...,45.746023,4.823894,"[12.04, 12.71, 12.54, 10.15, 11.62]","[partiellement nuageux, couvert, légère pluie,...",46.159113,-1.152043
873,35,"5e arr., Lyon",MiHotel Tour Rose,https://www.booking.com/hotel/fr/mihotel-tour-...,"8,4",Vous pouvez bénéficier d'une réduction Genius ...,45.762725,4.826820,"[12.04, 12.71, 12.54, 10.15, 11.62]","[partiellement nuageux, couvert, légère pluie,...",46.159113,-1.152043


In [45]:
# sauvegarde du dataframe en csv pour envoie dans un s3
csv = df.to_csv()
df.to_csv("hotel.csv")

In [6]:
# création d'une session aws
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=ACCESS_KEY_SECRETE)
s3 = session.resource("s3")

In [7]:
# création d'un bucket s3
bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [8]:
# ajout du fichier csv dans le compartiment bucket s3
put_object = bucket.put_object(Key="hotel.csv", Body=csv)

In [9]:
# recuperation du fichier csv dans un s3
df = pd.read_csv(f's3://{BUCKET_NAME}/hotel.csv')

In [42]:
df_temps = df[["id", "ville", "lat", "long", "temperature", "temps"]]

In [43]:
df_temps.drop_duplicates("id", inplace=True)
df_temps.reset_index(drop=True, inplace=True)

C:\Users\jerem\AppData\Local\Temp\ipykernel_24592\3048149287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps.drop_duplicates("id", inplace=True)


In [44]:
df_temps["moyenne"] = df_temps["temperature"].apply(lambda x : round((sum(x) / len(x)),2) )

C:\Users\jerem\AppData\Local\Temp\ipykernel_24592\4131000444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps["moyenne"] = df_temps["temperature"].apply(lambda x : round((sum(x) / len(x)),2) )


In [49]:
jour = list(range(1,6))
df_temps['jour'] = [jour for _ in range(len(df_temps))]

C:\Users\jerem\AppData\Local\Temp\ipykernel_24592\158154597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps['jour'] = [jour for _ in range(len(df_temps))]


In [51]:
df_temps = df_temps.apply(pd.Series.explode)

In [53]:
df.drop(["lat", "long", "temperature", "temps"], inplace=True, axis=1)

In [54]:
# engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOSTNAME}/postgres", echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)

In [55]:
df.to_sql("hotel", engine)


2023-01-04 08:03:55,909 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotel")
2023-01-04 08:03:55,909 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:03:55,910 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotel")
2023-01-04 08:03:55,910 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:03:55,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:03:55,912 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	"index" BIGINT, 
	id BIGINT, 
	ville TEXT, 
	hotel TEXT, 
	url TEXT, 
	note TEXT, 
	description TEXT, 
	latitutde FLOAT, 
	longitude FLOAT
)


2023-01-04 08:03:55,914 INFO sqlalchemy.engine.Engine [no key 0.00187s] ()
2023-01-04 08:03:55,916 INFO sqlalchemy.engine.Engine CREATE INDEX ix_hotel_index ON hotel ("index")
2023-01-04 08:03:55,916 INFO sqlalchemy.engine.Engine [no key 0.00059s] ()
2023-01-04 08:03:55,917 INFO sqlalchemy.engine.Engine COMMIT
2023-01-04 08:03:55,918 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:03:55,

875

In [56]:
df_temps.to_sql("temps", engine)

2023-01-04 08:03:58,844 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("temps")
2023-01-04 08:03:58,844 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:03:58,845 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("temps")
2023-01-04 08:03:58,846 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:03:58,847 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:03:58,847 INFO sqlalchemy.engine.Engine 
CREATE TABLE temps (
	"index" BIGINT, 
	id BIGINT, 
	ville TEXT, 
	lat FLOAT, 
	long FLOAT, 
	temperature FLOAT, 
	temps TEXT, 
	moyenne FLOAT, 
	jour BIGINT
)


2023-01-04 08:03:58,848 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2023-01-04 08:03:58,849 INFO sqlalchemy.engine.Engine CREATE INDEX ix_temps_index ON temps ("index")
2023-01-04 08:03:58,849 INFO sqlalchemy.engine.Engine [no key 0.00039s] ()
2023-01-04 08:03:58,850 INFO sqlalchemy.engine.Engine COMMIT
2023-01-04 08:03:58,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:03:58,853

175

In [57]:
conn = engine.connect()

In [58]:
ville = text("SELECT * FROM temps ORDER BY moyenne DESC LIMIT 25")


In [59]:
hotel_1 = text("SELECT * FROM hotel WHERE id = 21 ORDER BY note DESC LIMIT 20")
hotel_2 = text("SELECT * FROM hotel WHERE id = 20 ORDER BY note DESC LIMIT 20")
hotel_3 = text("SELECT * FROM hotel WHERE id = 35 ORDER BY note DESC LIMIT 20")
hotel_4 = text("SELECT * FROM hotel WHERE id = 2 ORDER BY note DESC LIMIT 20")
hotel_5 = text("SELECT * FROM hotel WHERE id = 1 ORDER BY note DESC LIMIT 20")

In [32]:
# result_hotel.fetchall()

In [60]:
hotel_1 = pd.read_sql_query(hotel_1, conn)
hotel_2 = pd.read_sql_query(hotel_2, conn)
hotel_3 = pd.read_sql_query(hotel_3, conn)
hotel_4 = pd.read_sql_query(hotel_4, conn)
hotel_5 = pd.read_sql_query(hotel_5, conn)
hotel_viz = pd.concat([hotel_1,hotel_2,hotel_3,hotel_4,hotel_5], ignore_index=False)
ville_viz = pd.read_sql_query(ville, conn)

2023-01-04 08:04:07,724 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 21 ORDER BY note DESC LIMIT 20
2023-01-04 08:04:07,726 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ()
2023-01-04 08:04:07,728 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 20 ORDER BY note DESC LIMIT 20
2023-01-04 08:04:07,729 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ()
2023-01-04 08:04:07,731 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 35 ORDER BY note DESC LIMIT 20
2023-01-04 08:04:07,731 INFO sqlalchemy.engine.Engine [generated in 0.00055s] ()
2023-01-04 08:04:07,734 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 2 ORDER BY note DESC LIMIT 20
2023-01-04 08:04:07,734 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
2023-01-04 08:04:07,737 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 1 ORDER BY note DESC LIMIT 20
2023-01-04 08:04:07,737 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ()
2023-01-04 08:04:

In [67]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(
    ville_viz,
    lat='lat',
    lon='long',
    color='temperature',
    size='temperature',
    color_continuous_scale=px.colors.diverging.Picnic,
    size_max=30,
    zoom=4.7,
    hover_name='ville',
    hover_data={
        'lat': False,
        'long': False,
        'moyenne': True        
        },
    animation_frame='jour'
)

fig.update_layout(width = 1500, height = 800, template='plotly_dark' ,title='Les 5 meilleurs ville sur un moyenne de température')
fig.show()

In [69]:
hotel_viz

,index,id,ville,hotel,url,note,description,latitutde,longitude
0,511,21,"Toulouse Sud-Est, Toulouse","La loge du Minotaure, cosy, parking et bureaux",https://www.booking.com/hotel/fr/59m2-parking-...,"9,6",Doté d'une aire de jeux pour enfants et d'une ...,43.575154,1.473728
1,500,21,"Toulouse Rive Gauche, Toulouse",LA CARTOUCHERIE,https://www.booking.com/hotel/fr/la-cartoucher...,"9,4",Vous pouvez bénéficier d'une réduction Genius ...,43.601721,1.408088
2,509,21,"Centre de Toulouse, Toulouse",Les Loges des Chalets,https://www.booking.com/hotel/fr/les-loges-des...,"9,2","Situé à 5.1 km du Stadium de Toulouse, l'établ...",43.614051,1.447940
3,514,21,"Minimes - Barrière de Paris, Toulouse",Large and modern flat just nearby Ponts-Jumeau...,https://www.booking.com/hotel/fr/large-and-mod...,"8,9",Le Large and modern flat just nearby Ponts-Jum...,43.613013,1.421539
4,503,21,"Centre de Toulouse, Toulouse",Le Bonrepos - Appartement 2 chambres clim & pa...,https://www.booking.com/hotel/fr/appartement-d...,"8,7",Vous pouvez bénéficier d'une réduction Genius ...,43.609439,1.453500
...,...,...,...,...,...,...,...,...,...
15,10,1,Le Mont-Saint-Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,"7,9",Vous pouvez bénéficier d'une réduction Genius ...,48.616263,-1.510906
16,5,1,Le Mont-Saint-Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"7,8",Vous pouvez bénéficier d'une réduction Genius ...,48.617587,-1.510396
17,13,1,Ardevon,Auberge de la Baie,https://www.booking.com/hotel/fr/auberge-de-la...,"7,8",Vous pouvez bénéficier d'une réduction Genius ...,48.615993,-1.488253
18,12,1,Le Mont-Saint-Michel,Hôtel la Croix Blanche,https://www.booking.com/hotel/fr/ha-el-la-croi...,"7,6",Situé au cœur du village médiéval du Mont-Sain...,48.635734,-1.509861


In [73]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(
    hotel_viz,
    lat='latitutde',
    lon='longitude',
    color='note',
    size='note',
    color_continuous_scale=px.colors.diverging.Picnic,
    size_max=30,
    zoom=4.7,
    hover_name='ville',
    hover_data={
        'latitutde': False,
        'longitude': False,
        'hotel': True,
        'note': True    
        }, 
)

fig.update_layout(width = 1500, height = 800, template='plotly_dark' ,title='Les 5 meilleurs ville sur un moyenne de température')
fig.show()

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [24]:
hotel_viz

,index,id,ville,hotel,url,note,description,latitutde,longitude
0,686,28,Uzès,Maison De La Bourgade,https://www.booking.com/hotel/fr/maison-de-la-...,"9,8","Situé à Uzès, à 31 km du parc des expositions ...",44.009668,4.418032
1,689,28,Uzès,AU BORD DE L'ALZON,https://www.booking.com/hotel/fr/au-bord-de-l-...,"9,8",Vous pouvez bénéficier d'une réduction Genius ...,43.995238,4.421976
2,680,28,Uzès,"Les Canisses, magnifique appartement avec terr...",https://www.booking.com/hotel/fr/magnifique-ap...,"9,7","Situé à Uzès, à 32 km du parc des expositions ...",44.013545,4.417073
3,682,28,Uzès,UZES DECOUVERTES,https://www.booking.com/hotel/fr/uzes-decouver...,"9,7","Situé à Uzès, à moins de 32 km du parc des exp...",44.018242,4.418825
4,681,28,Uzès,Bon Jour Uzès,https://www.booking.com/hotel/fr/bon-jour-uzes...,"9,6",Vous pouvez bénéficier d'une réduction Genius ...,44.012197,4.419125
...,...,...,...,...,...,...,...,...,...
15,834,34,La Martre,Château et Golf De Taulane,https://www.booking.com/hotel/fr/chateau-et-go...,"8,4",Vous pouvez bénéficier d'une réduction Genius ...,43.773934,6.619480
16,828,34,Aiguines,Les Bastides de Chanteraine,https://www.booking.com/hotel/fr/les-bastides-...,"8,3","Situé à Aiguines, l'établissement Les Bastides...",43.762734,6.219182
17,829,34,Bauduen,Domaine de Majastre,https://www.booking.com/hotel/fr/domaine-de-ma...,"8,1",Située à 9 km de Bauduen et à 24 km de Moustie...,43.681537,6.225509
18,827,34,Aiguines,Gabrielle hôtel,https://www.booking.com/hotel/fr/logis-altitud...,"7,9","Situé au cœur du parc national du Verdon, l'Ga...",43.776331,6.242523


In [25]:
fig8 = px.scatter_mapbox(hotel_viz, lat="latitutde", lon="longitude",color='note', zoom=5, mapbox_style="carto-positron",color_continuous_scale='Reds')
fig8.show()